## FIFA'21 Dataset

#### The major aim of this project is to clean the  messy, raw dataset for exploring

In [1]:
#importing libraries
import pandas as pd
import numpy as np

In [469]:
#reading the dataset to a dataframe
df = pd.read_csv('fifa21 raw data v2.csv')

#the first five records of the data
df.head(5)

/var/folders/73/dfnlcqg57w7btn76pt1r17cm0000gp/T/ipykernel_6865/2715229434.py:2: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('fifa21 raw data v2.csv')


,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,\n\n\n\nFC Barcelona,...,Medium,Low,5 ★,85,92,91,95,38,65,771
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,\n\n\n\nJuventus,...,High,Low,5 ★,89,93,81,89,35,77,562
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,...,Medium,Medium,3 ★,87,92,78,90,52,90,150
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,\n\n\n\nManchester City,...,High,High,4 ★,76,86,93,88,64,78,207
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,...,High,Medium,5 ★,91,85,86,94,36,59,595


In [470]:
print(f"This is the Dataset information :\n")
df.info()

This is the Dataset information :

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                18979 non-null  int64 
 1   Name              18979 non-null  object
 2   LongName          18979 non-null  object
 3   photoUrl          18979 non-null  object
 4   playerUrl         18979 non-null  object
 5   Nationality       18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Club              18979 non-null  object
 10  Contract          18979 non-null  object
 11  Positions         18979 non-null  object
 12  Height            18979 non-null  object
 13  Weight            18979 non-null  object
 14  Preferred Foot    18979 non-null  object
 15  BOV               18979 non-null  int64 
 16  Best Position     18979

In [471]:
#the number of rows and columns 
rows, columns = df.shape
print(f"There are {rows} rows and {columns} columns in the dataset")

There are 18979 rows and 77 columns in the dataset


# Data Cleaning

#### Issue 1. Converting the header names to lower case for uniformity

In [472]:
# Converting all the header name into lower case
df.columns = [x.lower() for x in df.columns]

#renaming the column 'ova'
df.rename(columns = {'↓ova': 'ova'}, inplace = True)

# remove special character
df.columns = df.columns.str.replace(' ', '_')

In [473]:
df.columns

Index(['id', 'name', 'longname', 'photourl', 'playerurl', 'nationality', 'age',
       'ova', 'pot', 'club', 'contract', 'positions', 'height', 'weight',
       'preferred_foot', 'bov', 'best_position', 'joined', 'loan_date_end',
       'value', 'wage', 'release_clause', 'attacking', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'skill', 'dribbling',
       'curve', 'fk_accuracy', 'long_passing', 'ball_control', 'movement',
       'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
       'power', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'mentality', 'aggression', 'interceptions', 'positioning', 'vision',
       'penalties', 'composure', 'defending', 'marking', 'standing_tackle',
       'sliding_tackle', 'goalkeeping', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes', 'total_stats',
       'base_stats', 'w/f', 'sm', 'a/w', 'd/w', 'ir', 'pac', 'sho', 'pas',
       'dri', 'def', '

#### Issue 2. Dealing with the `club` Column 

In [474]:
#reviewing the sample of club column
df['club'].sample()

1779    \n\n\n\nClub Brugge KV
Name: club, dtype: object

In [475]:
#removing the new lines
df['club'] = df['club'].apply(lambda x: x.strip())

#### Issue 3 .  Dealing with the `contract` Column 

In [476]:
df['contract'] = df['contract'].apply(lambda x: x.replace(' ~ ','-'))

- I noticed that some players are under contract, free,  while others are on loan.
- In order to keep the column type consistent, we classify players under contract as active while classifying those on loan as such  unbiased parties

In [477]:
#function to define the terms of the contract 
def contract_terms(item):
    if '-' in item:
        return 'active'
    if 'On Loan' in item:
        return 'loan'
    if 'Free' in item:
        return 'free'

In [478]:
df['contract_terms'] = df['contract'].apply(contract_terms)

#### Issue 4 .  Dealing with the `Joined` Column 

- Based on the 'Joined' column, we can check which players have been playing at a club for more than 10 years!

In [479]:
#converting the joined date to pandas datetime and extracting the year
df['previous_year'] = pd.to_datetime(df.joined).dt.year

#changing the datatype to int for futher processing
df['previous_year'] = df['previous_year'].astype('int')

In [480]:
#importing the required library to get the present year
import datetime


df['current_year'] = datetime.datetime.now().year


#changing the datatype to int for futher processing
df['current_year'] = df['current_year'].astype('int')

In [481]:
#the number of years the players has spent in a paricular club
df['players_club_year'] = df['current_year'] - df['previous_year']

#### Issue 5. Dealing with `Value`, `Wage`, `Release Clause` Columns

In [482]:
#uniform function to deal with the messiness
def values(item):
    try:
        item = item.replace('€', '').strip()
        if 'K' in item:
            item = float(item.replace('K', ''))
            item  = item * 1000
        elif 'M' in item:
            item = float(item.replace('M',''))
            item = item * 1000000
        else:
            item = float(item)
        return round(item)
    
    except:
        return 0

In [483]:
#applying the function on the columns
df['value_'] = df['value'].apply(values)
df['wage_'] = df['wage'].apply(values)
df['release_clause_'] = df['release_clause'].apply(values)

#### Issue 6. Dealing with `Height` and `Weight` Columns

In [484]:
# function to deal with the Height column as some values were given in 'cm' and some in 'feet&inches'
def parse_cm(item):
    if 'cm' in item:
        item = item.replace('cm', '')
    else: 
        item = item.replace('"','')
        value = item.split("\'")
        item = (int(value[0]) * 30.48) + (int(value[1]) * 2.54)
    return int(item)

In [486]:
#applying the function
df['height (cm)'] = df['height'].apply(parse_cm)

In [487]:
# function to deal with the Weight column as some values were given in 'kg' and some in 'lbs(pounds)'
def parse_kg(item):
    if 'kg' in item:
        item = item.replace('kg', '')
    else:
        value = item.replace('lbs', '')
        item = int(value) * 0.453592
    return round(int(item))

In [488]:
#applying the function
df['weight (kg)'] = df['weight'].apply(parse_kg)

#### Issue 7. Dealing with `w/f `, `sm`,  and the   `ir` columns

In [489]:
# function to remove the 'star' characters
def remove_star(item):
    item = str(item)
    item = int(item.replace('★', '').strip())
    return item

In [490]:
#applying the function
df['w/f_'] = df['w/f'].apply(remove_star)
df['sm_'] = df['sm'].apply(remove_star)
df['ir_'] = df['ir'].apply(remove_star)

#### Issue 8. Dealing with the  `hits` column

In [491]:
df['hits_'] = df['hits'].apply(values)

### Renaming and dropping the irrelevant  columns

In [492]:
columns = ['id', 'name', 'photourl', 'playerurl', 'joined',  'loan_date_end', 'value', 'wage', 'height', 'weight', 
           'release_clause', 'w/f', 'a/w','ir', 'hits']

In [493]:
#dropping the columns
df = df.drop(columns = columns , axis=1)

In [494]:
dict = {'wage_' : 'wage',
        'release_clause_': 'release_clause',
        'value_': 'value',
        'w/f_' : 'w/f',
        'sm_' : 'sm',
        'ir_' : 'ir',
        'hits_': 'hits',
       'height (cm)': 'height (cm)',
       'weight (kg)':'weight (kg)'}

In [495]:
# call rename () method
df.rename(columns = dict, inplace=True)

- Since all the cleaning is done, I would be exporting to a new csv for exploration

In [496]:
df_cleaned = df.to_csv('cleaned_fifa_data.csv')